In [146]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer

In [147]:
df = pd.read_csv('train.csv')[['Age','Fare','SibSp','Parch','Survived']]
df.head()

,Age,Fare,SibSp,Parch,Survived
0,22.0,7.2500,1,0,0
1,38.0,71.2833,1,0,1
2,26.0,7.9250,0,0,1
3,35.0,53.1000,1,0,1
4,35.0,8.0500,0,0,0


In [148]:
df.isnull().sum()

Age         177
Fare          0
SibSp         0
Parch         0
Survived      0
dtype: int64

In [149]:
df.dropna(inplace=True)  # Dropping the NULL values

In [150]:
df['family'] = df['SibSp'] + df['Parch']
df.head()

,Age,Fare,SibSp,Parch,Survived,family
0,22.0,7.2500,1,0,0,1
1,38.0,71.2833,1,0,1,1
2,26.0,7.9250,0,0,1,0
3,35.0,53.1000,1,0,1,1
4,35.0,8.0500,0,0,0,0


In [151]:
df.drop(columns=['SibSp','Parch'],inplace=True)
df.head()

,Age,Fare,Survived,family
0,22.0,7.2500,0,1
1,38.0,71.2833,1,1
2,26.0,7.9250,1,0
3,35.0,53.1000,1,1
4,35.0,8.0500,0,0


### Model Building :

In [152]:
X = df.drop(columns=['Survived'])
y = df['Survived']

In [153]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [154]:
X_train.head()

,Age,Fare,family
328,31.0,20.5250,2
73,26.0,14.4542,1
253,30.0,16.1000,1
719,33.0,7.7750,0
666,25.0,13.0000,0


In [155]:
# Without binarization
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

In [156]:
print("Accuracy Score : ",accuracy_score(y_test,y_pred))
print("Cross Val Score :",np.mean(cross_val_score(DecisionTreeClassifier(),X,y,cv=10,scoring='accuracy')))

Accuracy Score :  0.6223776223776224
Cross Val Score : 0.6457550860719875


### Binarization :

In [157]:
from sklearn.preprocessing import Binarizer

In [158]:
trf = ColumnTransformer([
                        ('bin',Binarizer(copy=False),['family'])
                        ],remainder='passthrough')

### NOTE :

`threshold` ==> Feature values below or equal to this are replaced by 0, above it by 1. Default :`0.0`

`copy` ==> Set to "False" to perform inplace binarization and avoid a copy. Default : `True`


In [159]:
X_train_trf = trf.fit_transform(X_train)
X_test_trf = trf.transform(X_test)

In [160]:
pd.DataFrame(X_train_trf,columns=['family','Age','Fare'])

,family,Age,Fare
0,1.0,31.0,20.5250
1,1.0,26.0,14.4542
2,1.0,30.0,16.1000
3,0.0,33.0,7.7750
4,0.0,25.0,13.0000
...,...,...,...
566,1.0,46.0,61.1750
567,0.0,25.0,13.0000
568,0.0,41.0,134.5000
569,1.0,33.0,20.5250


In [161]:
clf = DecisionTreeClassifier()
clf.fit(X_train_trf,y_train)
y_pred2 = clf.predict(X_test_trf)

In [162]:
print("Accuracy Score : ",accuracy_score(y_test,y_pred2))

X_trf = trf.fit_transform(X)
print("Cross Val Score :",np.mean(cross_val_score(DecisionTreeClassifier(),X_trf,y,cv=10,scoring='accuracy')))

Accuracy Score :  0.6293706293706294
Cross Val Score : 0.6261737089201878
